<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [14]:
import os
import io
import sys
import re
import csv
import pandas as pd
from tqdm import tqdm_notebook, tnrange
from datetime import datetime

In [30]:
re_From = re.compile(r'^\s?From\:\s?(.*)')
re_From2 = re.compile(r'^F.S. 119.071 - Security .*From\:\s(.*)')
re_Sent = re.compile(r'^Sent\:\s?(.*)')
re_Sent2 = re.compile(r'^Date\:\s?(.*)')
re_To = re.compile(r'^To\:\s?(.*)')
re_CC = re.compile(r'^C[Cc]\:\s?(.*)')
re_Subject = re.compile(r'^Sub\s?j\s?ect\:\s?(.*)')
re_Subject2 = re.compile(r'^Title\:\s?(.*)')
re_Attachments = re.compile(r'^Attachments\:\s?(.*)')
re_Importance = re.compile(r'Importance\:\s?(.*)')

re_isThread = re.compile(r'isThread\:\s?(.*)')
re_isAutoMessage = re.compile(r'isAutoMessage\:\s?(.*)')
re_hasAllCapLine = re.compile(r'hasAllCapLine\:\s?(.*)')
re_hasBadDate = re.compile(r'hasBadDate\:\s?(.*)')
re_isDisplacement = re.compile(r'isDisplacement\:\s?(.*)')
        
re_page = re.compile(r'^Page\s\d+$')
re_page2 = re.compile(r'^Page\s\d+\sof\s\d+$')
re_page3 = re.compile(r'^\d+\-\d+$')

re_tail = re.compile(r'^Sent\sfrom\s.*\siPad')
re_tail2 = re.compile(r'^Sent\sfrom\s.*\siPhone')
re_tail3 = re.compile(r'^On\s\w+\,\s\w+\swrote\:')
re_tail4 = re.compile(r'^On\s.+\,\s[0-9]{4},\sat\s.+\,.+')
re_tail5 = re.compile(r'^Begin\sforwarded\smessage\:')



tails = ['Sent from my iPhone',
         "Sent from Paige's iPad",
        'On \w+\, \w+ wrote:',
        'Text messages of Karen Jumonville To/From Adam Corey',
         'On Jun 1, 2013, at 8:21 AM, "Favors Thompson, Anita"',
        'Begin forwarded message:']

re_lowercase = re.compile(r'^[a-z].*')

In [31]:
# test
re_tail3.findall("Sent from K's iPhone")
re_tail5.findall(tails[5])

['Begin forwarded message:']

In [32]:
def split_emails(inputpath, outputpath, filecount = None):
#     f_in = io.open(inputpath, encoding='utf8',errors='ignore')
    f_in = open(inputpath, encoding='utf8')

    if filecount == None:
        filecount = 0   # len(os.listdir(outputpath)) # files are named by numbers
    email = []
#     email.append(f_in.readline().strip('\f'))
    for line in f_in.readlines():
        line = line.strip('\f').strip(' ')
        if re_From.match(line) or re_From2.match(line):
            if email != []:
                with open(outputpath + '/' + str(filecount) + '.txt', 'w+', encoding = 'utf-8') as f_out:
                    for l in email:
                        f_out.write(l)
            email=[]
            filecount = filecount + 1
        email.append(line)
    f_in.close
    return filecount

In [33]:
# inputpath = './email_txt/'
# outputpath = './split_txt/' 
# inputpath = './redacted_email_txt/'
# outputpath = './redacted_split_txt/' 
# inputpath = './unredacted_email_txt/'
# outputpath = './unredacted_split_txt/' 
inputpath = './old_email_txt/'
outputpath = './old_split_txt/' 

filecount = 0
for inputfile in tqdm_notebook(os.listdir(inputpath)):
#     print('processing '+inputpath+inputfile)
    filecount = split_emails(inputpath+inputfile, outputpath, filecount-1)
#     if filecount > 10300:
#         break
    

FileNotFoundError: [Errno 2] No such file or directory: './old_email_txt/'

In [34]:
def correct_line(inputpath, outputpath=''):
    if outputpath=='':
        outputpath = inputpath
    new_file = []
    with open(inputpath, 'r', encoding='utf8') as f:
        flag = 0
        body = []
        for line in f.readlines():
            line = re.sub('\f', '', line)
            line = line.strip()
            
            if re_tail.match(line) or re_tail2.match(line) or re_tail3.match(line) or re_tail4.match(line) or re_tail5.match(line):
                break
            if re_page.match(line) or re_page2.match(line) or re_page3.match(line):
                continue
            if re_From.match(line) or re_From2.match(line) or re_Sent.match(line) or re_Sent2.match(line) or re_To.match(line) or re_CC.match(line):
                if body != []:
                    temp_line = ' '.join(body)
                    new_file.append(temp_line)
                body = []
                body.append(line)
            elif re_Subject.match(line):
                flag = 1
                temp_line = ' '.join(body)
                body = []
                new_file.append(temp_line)
                body.append(line)
            elif re_Attachments.match(line) and flag == 1:
                flag = 2
                temp_line = ' '.join(body)
                body = []
                body.append(line)
                new_file.append(temp_line)
            elif re_Attachments.match(line) and flag == 0:
                flag = 3
                temp_line = ' '.join(body)
                body = []
                new_file.append(temp_line)
                body.append(line)
#                 print(line)
#                 print(re_Attachments.match(line))
#                 print( re_Attachments.findall(line))
                print('no title but attachment in file'+inputpath)
#                 return
            else:
                body.append(line)
        if flag == 0:
            flag2 = 0
            new_file.append(body[0])
            for line in body[1:]:
                if ('<' in line or '>' in line or '(' in line or ')' in line or ',' in line or ';' in line or line == '') and flag2 == 0:
                    new_file[-1] = new_file[-1] + ' ' + line
                elif flag2 == 1:
                    new_file.append(line)
                else:
                    flag2 = 1
                    new_file.append('Subject: ' + line)
        else:
            for line in body:
                new_file.append(line)
        

    with open(outputpath, 'w+', encoding = 'utf-8') as f_out:
        for line in new_file:
            f_out.write(line + '\n')

In [35]:
# inputpath = './split_txt/' 
# inputpath = './redacted_split_txt/' 
# inputpath = './unredacted_split_txt/' 
inputpath = './old_split_txt/' 

abnormal_email_id = []
for name in tqdm_notebook(os.listdir(inputpath)):
    try:
        correct_line(inputpath+'/'+name)
    except:
        abnormal_email_id.append(int(name[:-4]))
        print(name)
#         os.remove(inputpath+'/'+name)

FileNotFoundError: [Errno 2] No such file or directory: './old_split_txt/'

In [49]:
# test
# correct_line('./unredacted_split_txt/66012.txt')

no title but attachment in file./unredacted_split_txt/66012.txt


---
## Header notes
* We remove "Body" from the header messages
* Create columns for mail Id, 'isThread', 'isAutoMessage', 'hasAllCapLine', 'hasBadDate', 'isDisplacement'

In [46]:
def get_headers(inputpath, outputpath=""):
    dictionary = {}
    with open(inputpath, "r", encoding="utf8") as f:
        list_from = []
        list_sent = []
        list_to = []
        list_cc = []
        list_subject = []
        list_attachments = []
        list_body = []
        list_importance = []
        list_isThread = False
        list_isAutoMessage = False
        list_hasAllCapLine = False
        list_hasBadDate = False
        list_isDisplacement = False

        for line in f.readlines():
            # \f is a special character (dot)
            line = re.sub("\f", "", line)
            line = line.strip()
            if re_From.match(line):
                list_from.append(re_From.findall(line)[0])
            elif re_Sent.match(line) or re_Sent2.match(line):
                if re_Sent.match(line):
                    list_sent.append(re_Sent.findall(line)[0])
                else:
                    list_sent.append(re_Sent2.findall(line)[0])

            elif re_To.match(line):
                list_to.append(re_To.findall(line)[0])
            elif re_CC.match(line):
                list_cc.append(re_CC.findall(line)[0])
            elif re_Subject.match(line) or re_Subject2.match(line):
                if re_Subject.match(line):
                    list_subject.append(re_Subject.findall(line)[0])
                else:
                    list_subject.append(re_Subject2.findall(line)[0])

            elif re_Attachments.match(line):
                list_attachments.append(re_Attachments.findall(line)[0])
            elif re_Importance.match(line):
                list_importance.append(re_Importance.findall(line)[0])
                
            elif re_isDisplacement.match(line):
                list_isDisplacement = True             
            elif re_isThread.match(line):
                list_isThread = True             
            elif re_isAutoMessage.match(line):
                list_isAutoMessage = True       
            elif re_hasAllCapLine.match(line):
                list_hasAllCapLine = True           
            elif re_hasBadDate.match(line):
                list_hasBadDate = True

            else:
                if line != "Body:":
                    line = re.sub(
                        "[^A-Za-z0-9\s\(\)\/\\\&\?\!\,\.\-\@\<\>\:\+\%\;]", "", line
                    )
                    list_body.append(line)

    dictionary["From"] = list_from
    dictionary["Sent"] = list_sent
    dictionary["To"] = list_to
    dictionary["CC"] = list_cc
    dictionary["Subject"] = list_subject
    dictionary["Attachments"] = list_attachments
    dictionary["Body"] = list_body
    dictionary["Importance"] = list_importance
    dictionary["isDisplacement"] = list_isDisplacement
    dictionary["isThread"] = list_isThread
    dictionary["isAutoMessage"] = list_isAutoMessage
    dictionary["hasAllCapLine"] = list_hasAllCapLine
    dictionary["hasBadDate"] = list_hasBadDate

    return dictionary

In [47]:
# inputpath = './split_txt/' 
# inputpath = './redacted_split_txt/' 
# inputpath = './unredacted_split_txt/' 
inputpath = './old_split_txt/' 
inputpath = './test_output/'
l = []
for name in tqdm_notebook(os.listdir(inputpath)):
    if name != '.DS_Store':
        l.append(get_headers(inputpath+'/'+name))
len(l)

<ipython-input-47-00a5ec6fe057>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm_notebook(os.listdir(inputpath)):


  0%|          | 0/71145 [00:00<?, ?it/s]

71145

In [48]:
csv_columns = ['From','Sent','To','CC','Subject','Attachments','Importance','Body',
               'isDisplacement','isThread','isAutoMessage','hasAllCapLine','hasBadDate'
              ]
# with open('./output.csv', 'w',encoding='utf8') as csvfile:
# with open('./redacted_output.csv', 'w',encoding='utf8') as csvfile:
# with open('./unredacted_output.csv', 'w',encoding='utf8') as csvfile:
with open('./old_output.csv', 'w',encoding='utf8') as csvfile:

    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in l:
        writer.writerow(data)

In [27]:
# df = pd.read_csv('output.csv')
# df_redacted = pd.read_csv('redacted_output.csv')
# df_unredacted = pd.read_csv('unredacted_output.csv')
df_old = pd.read_csv('old_output.csv')

# df.head()
# df_redacted.head()
# df_unredacted.head()
df_old.head()
df_old['Body'].head(50)

0     ['Response.pdf', 'Brooks, attached is our resp...
1                                       ['Good by me!']
2     ['Kim,', 'See comments in blue regarding the p...
3     ['Hi Alison,', 'Dont apologize, I know its a l...
4     ['Highly recommend you take a moment out of yo...
5     ['I havent heard anything yet.', 'Thanks for a...
6     ['Dear Friend,', 'As part of our 2016 Family F...
7     ['scof, gil.zier@talgov.com, curs.richardson@t...
8     ['Chad,', 'Would you please outline your antic...
9     ['Hello All,', 'Since all of you went to Bould...
10    ['Venus and Gary,', 'Thank you for meeng with ...
11    ['Dear EZDA Board of Commissioners:', 'You are...
12    ['Good Morning Mr. Thompson,', 'We have not re...
13                         ['', 'isDisplacement: True']
14    ['Good morning,', 'As a Ravines resident, my f...
15    ['ron@sachsmedia.com, communityinfo@leonschool...
16    ['Ryan, while I was at the Edison today I noti...
17    ['Thats ne.  Please forward the informaon 

In [64]:
# df = df_redacted.copy()
# df = df_unredacted.copy()
df = df_old.copy()

dates = df['Sent'].values.tolist()
for i in range(len(dates)):
    dates[i]= eval(dates[i])
new_dates = []
for idx, date in enumerate(tqdm_notebook(dates)):
    
    if date != []:
        dt = date[0]
        try:
            new_date = datetime.strptime(dt, "%A, %B %d, %Y %I:%M %p") 
        except:
            try:
                new_date = datetime.strptime(dt, "%A, %B %d, %Y %I: %M %p") 
            except:
                try:
                    new_date = datetime.strptime(dt, "%A %B %d, %Y %I:%M %p") 
                except:
                    try:
                        new_date = datetime.strptime(dt, "%B %d, %Y %I:%M:%S %p %Z") 
                    except:
                        try:
                            new_date = datetime.strptime(dt, "%B %d, %Y, %I:%M:%S %p %Z") 
                        except:
                            try:
                                new_date = datetime.strptime(dt, "%B %d, %Y %H:%M:%S %Z") 
                            except:
                                try:
                                    new_date = datetime.strptime(dt, "%A. %B %d, %Y %I:%M %p") 
                                except:
                                    try:
                                        new_date = datetime.strptime(dt, "%B %d, %Y %I:%M %p") 
                                    except:
                                        try:
                                            new_date = datetime.strptime(dt, "%A. %B %d. %Y %I:%M %p") 
                                        except:
                                            try:
                                                new_date = datetime.strptime(dt, "%A , %B %d, %Y %I:%M %p") 
                                            except:
                                                try:
                                                    new_date = datetime.strptime(dt, "%A, %B %d. %Y %I:%M %p") 
                                                except:
                                                    try:
                                                        new_date = datetime.strptime(dt, "%A, %B %d, %Y %I:%M:%S %p") 
                                                    except:
                                                        try:
                                                            new_date = datetime.strptime(dt, "%A, %B %d, %Y %I:%M:%S %p Eastern Time (US Canada)") 
                                                        except:
                                                            try:
                                                                new_date = datetime.strptime(dt, "%A, %B %d, %Y %I:%M")
                                                            except:
                                                                try:
                                                                    new_date = datetime.strptime(dt,"%B/%d/%Y %I:%M %p")
                                                                except:
                                                                    try:
                                                                        new_date = datetime.strptime(dt,"%B %d, %Y at %I:%M %p")
                                                                    except:
                                                                        try:
                                                                            new_date = datetime.strptime(dt, "%A, %B %d, %Y at %I:%M %p") 
                                                                        except:
                                                                            try:
                                                                                new_date = datetime.strptime(dt, "%A, %B %d, %Y %X %p (UTC-05:00) Eastern Time") 
                                                                            except:
                                                                                try:
                                                                                    new_date = datetime.strptime(dt, "%A, %B %d, %Y %X %p (UTC-05:00) Eastern Time (US & Canada)") 
                                                                                except:
                                                                                    new_date = datetime.strptime('00:00:00',"%X")
                                                                                    print('1',idx, dt)
                                        #                                 Wednesday, February 25, 2015 at 9:12 AM
                                        #                                 Friday, February 24, 6:47 PM
                                        #                                 Sunday, March 22, 2015 7:48:38 PM (UTC-05:00) Eastern Time
                                        #                                 Friday, August 05, 2016 5:34:39 PM (UTC-05:00) Eastern Time (US & Canada)
    else:        
        new_date = datetime.strptime('00:00:00',"%X")
        print(idx, dt)
    new_dates.append(new_date)
    
    
df['Sent2'] = new_dates

df = df.sort_values(by="Sent2")
# df_redacted = df.copy()
df_old = df.copy()

# df.to_csv("output2_time_identified.csv", header=True)

C:\Users\joeyz\anaconda3\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


1 63 Thu, 7 Jun 2012 12:02:55 +0000
1 72 Thu, 7 Jun 2012 12:02:55 +0000
1 165 August 30, 2012 9:26:16 EDT
1 169 August 30, 2012 9:26:16 EDT
1 184 August 30, 2012 9:26:16 EDT
1 216 August 30, 2012 9:26:16 EDT
224 Wednesday, September 12, 2012 2:22 PM
228 Wednesday, September 12, 2012 2:22 PM
1 625 07/18/2014 10:14 AM
1 647 07/18/2014 02:04 PM
1 658 07/18/2014 10:14 AM
1 741 Thu, 13 Dec 2012 17:50:34 +0000
1 748 Thu, 13 Dec 2012 17:50:34 +0000
1 755 Thu, 13 Dec 2012 17:50:34 +0000
1 758 07/18/2014 10:02 AM
1 764 Thu, 13 Dec 2012 17:50:34 +0000
1 773 Thu, 13 Dec 2012 17:50:34 +0000
1 784 Thu, 13 Dec 2012 17:50:34 +0000
1 794 Thu, 13 Dec 2012 17:50:34 +0000
1 828 Thu, 13 Dec 2012 17:50:34 +0000
1 838 Thu, 13 Dec 2012 17:50:34 +0000
1 847 07/18/2014 10:02 AM
1 849 Thu, 13 Dec 2012 17:50:34 +0000
1 862 Thu, 13 Dec 2012 17:50:34 +0000
1 874 Thu, 13 Dec 2012 17:50:34 +0000
1 880 07/18/2014 10:14 AM
1110 Thursday, December 20, 2012 2:14 PM
1112 Thursday, December 20, 2012 2:33 PM
1 1123 Decembe

In [65]:
date2 = pd.to_datetime(df['Sent2'].values.tolist())
idx = date2 > datetime(2000,1,1)
df2 = df.iloc[idx]
df2 = df2.reset_index(drop=True)
# df2.to_csv('output3_with_good_time.csv')

In [66]:
df2.drop_duplicates(subset=['From','Sent2'],keep='first',inplace = True)
df2 = df2.reset_index(drop=True)
# df_redacted = df2.copy()
df_old = df2.copy()

# df2.to_csv('output4_without_dups.csv')
# df_redacted.to_csv('redacted_emails.csv')
df_unredacted.to_csv('unredacted_emails.csv')
df_old.to_csv('old_emails. csv')

df2

,From,Sent,To,CC,Subject,Attachments,Importance,Body,Sent2
0,"['Heiter, Deborah [mailto:Deborah.Heiter@talgo...","['Friday, January 20, 2012 4:41 PM']","[""Behenna, Bill; Branch, Cathy; Burn, Paula; C...",[],['Economic Development Target Issue Meeting'],[],[],"['', 'Deborah L. Heiter', 'Administrative Spec...",2012-01-20 16:41:00
1,['Beth Kirkland [mailto:bkirkland@taledc.com]'],"['Monday, January 23, 2012 9:32 AM']","[""'llaubscher@eflorida.com'""]","[""brook.pace@deo.myflorida.com; Kim Rivers (ki...",['City of Tallahassee meeting regarding SBCI'],[],[],"['Louis,', 'Thank you for meeting with us in F...",2012-01-23 09:32:00
2,"['Wiebler, Brian T.']","['Monday, February 27, 2012 10:04 AM']",[],[],['Rural Density'],[],[],[''],2012-02-27 10:04:00
3,"['Beaudoin, Mark']","['Monday, March 05, 2012 11:33 AM']","['Corey, Adam']","['English, Jim; Coe, Torn; Bell?ower, Adrienne']",['Sidewalk Cafe Agreement'],[],[],"['', 'Adam, thanks for the update regarding th...",2012-03-05 11:33:00
4,"['Wiebler, Brian T.']","['Tuesday, March 27, 2012 3:59 PM']","['Wiebler, Brian T.']",[],['Rural Density Comprehensive Plan Text Amendm...,[],[],"['', 'This is a reminder of the open house eve...",2012-03-27 15:59:00
...,...,...,...,...,...,...,...,...,...
13085,"['Campbell, Stacey']","['Friday, June 23, 2017 1:27 PM']",['Ryan Grindler'],"['Donahoe, Judy']",['RE: Request: Construction Warranties'],[],[],"['Perfect, thank you.']",2017-06-23 13:27:00
13086,['Gil Ziffer [mailto:gil@ziffberry.com]'],"['Wednesday, June 28, 2017 3:32 PM']","['Branch, Cathy <Cathy.Branch@talgov.com>']",[],['Fwd: 2013 AIP Forum White Paper_final (3).pdf'],[],[],"['', 'Gil Ziffer', '(850) 509-7886']",2017-06-28 15:32:00
13087,['Gil Ziffer [mailto:gil@ziffberry.com]'],"['Thursday, June 29, 2017 4:41 AM']","['Branch, Cathy <Cathy.Branch@talgov.com>']",[],['Fwd: 2013 AIP Forum White Paper_final (3).pdf'],[],[],"['', 'Subpoena']",2017-06-29 04:41:00
13088,['Gil Ziffer [mailto:gil@ziffberry.com]'],"['Thursday, June 29, 2017 5:35 AM']","['Branch, Cathy <Cathy.Branch@talgov.com>']",[],['Fwd: RE: Fwd:'],[],[],"['', 'Subpoena']",2017-06-29 05:35:00


In [ ]:
tails = ['Sent from my iPhone',
         "Sent from Paige's iPad",
        'On \w+\, \w+ wrote:',
        'Text messages of Karen Jumonville To/From Adam Corey']

In [ ]:
All the best,
Thanks,
Best,

In [ ]:
'Page 599 of 2062'
'Page 199'
'34-2'